In [81]:
#Trader v0.2.0
from modules.assets_handler import *
from pydoc import locate
from modules.klines_handler import OneMinuteCandlesAmount as OMCA, Transform

class Trader:
    # Common instantiation, trader and backing test
    def __init__(self, exchange_name, asset_symbol):
        # Asset Stuff
        self.exchange_name = exchange_name
        self.asset_symbol = asset_symbol
        self.Asset = AssetsHandler(exchange_name, asset_symbol)
        # Strategy
        self.strategy_name = self.Asset.operational_parameters()['strategy']['name']
        self.strategy_param = self.Asset.operational_parameters()['strategy']['parameters']
        self.strat_candle_interval = self.strategy_param['candle_intervalerval']
        self.Strategy = locate('modules.strategies.' + self.strategy_name)(self.strategy_param)        
        self.StrategyCandlesNum = self.Strategy.how_many_candles()        
        # Stop Loss
        self.stop_name = self.Asset.operational_parameters()['stop_loss']['name']
        self.stop_param = self.Asset.operational_parameters()['stop_loss']['parameters']
        self.stop_candle_interval = self.stop_param['candle_intervalerval']
        self.StopLoss = locate('modules.stop_loss.' + self.stop_name)(self.stop_param)
        self.StopCandlesNum = self.StopLoss.how_many_candles()
        # Order Handler (broker)
        self.OrderHandler = locate('modules.order_handler' + exchange_name.capitalize() + 'Handler')
        # Klines Handler
        self.KlinesHandler = locate('modules.klines_handler.' + exchange_name.capitalize() + 'FromDb')()
        
    def for_real(self):
        self.position = RealTradePosition(self.Asset.get_dataframe().iloc[0].position)
    
    def backing_test(self):
        self.position = BackingTestAssetPosition({'side': 'closed', 'size': 0.0, 'target_price': 0.0})
        all_1m_klines = self.KlinesHandler.all_latest_one_minute(self.asset_symbol)
        #TODO: Verificar se os tamanhos das 'slices' são menores que o número de klines de 1m
        strategy_1m_slice = OMCA(self.strat_candle_interval).total(self.StrategyCandlesNum)
        stop_1m_slice = OMCA(self.stop_candle_interval).total(self.StopCandlesNum)
        
        end_index = max(strategy_1m_slice, stop_1m_slice)
        strategy_start_index = end_index - strategy_1m_slice; stop_start_index = end_index - stop_1m_slice
        
        while end_index < len(all_1m_klines):
            
            if(position.side != 'closed'):
                klines_to_stop_analisys = Transform(
                    all_1m_klines[stop_start_index:end_index]).from_1m_to(self.stop_candle_interval)
                
                stop = StopLoss.verify(klines_to_stop_analisys, self.position)
    
                if (stop.is_true):
                    order = OrderHandler.execute(stop.order)
                    order_confirmation = OrderHandler.check(order)
                    Asset.update_position(order_confirmation)
    
                if(stop.update_target): self.position.target_price = stop.new_target_price
            
            klines_to_strategy_analisys = Transform(
                all_1m_klines[strategy_start_index:end_index]).from_1m_to(self.strat_candle_interval)

            trade = Strategy.verify(klines_to_strategy_analisys, self.position)

            if (trade.is_true):
                order = OrderHandler.execute(trade.order)
                order_confirmation = OrderHandler.check(order)
                Asset.update_position(order_confirmation)
            
            strategy_start_index+=1; stop_start_index+=1; end_index+=1

In [ ]:
#'order', tanto no stop quanto na estratégia, será um dicionário, para manter a simplicidade

In [2]:
exchange_name, asset_symbol = 'binance', 'BTCUSDT'

trader = Trader(exchange_name, asset_symbol)

In [7]:
trader.backing_test()

In [10]:
trader.position.show()

{'side': 'closed', 'size': 0.0, 'target_price': 0.0}

In [5]:
trader.strategy_1m_slice

6060

In [ ]:
from modules.klines_handler import OneMinuteCandlesAmount, Transform

n_stop = OneMinuteCandlesAmount(trader.stop_param['candle_interval']).total(trader.StopLoss.how_many_candles())

In [ ]:
n_stop

In [ ]:
trader.Strategy.how_many_candles()

In [ ]:
test_klines = trader.KlinesHandler.get_latest(asset_symbol, '1m', 100)
test_klines

In [6]:
trader.stop_param['candle_interval']

'1m'

In [ ]:
from modules.klines_handler import OneMinuteCandlesAmount, Transform

In [ ]:
OneMinuteCandlesAmount('1h').total(100)

In [ ]:
trader.Asset.get_dataframe().iloc[0].position

In [ ]:
trader.backing_test()

In [ ]:
StopCandlesNum = 'test'
StrategyCandlesNum
n_candles_to_stop_analisys = 

In [ ]:
trader.KlinesHandler.all_latest_one_minute

In [29]:
class Myclass:
    def __init__(self):
        self.my_name_is = self.__class__.__name__
    
    def who_amn_i(self):
        print('hello',self.my_name_is)

In [30]:
Myclass().who_amn_i()

hello Myclass


In [17]:
some_function('marcus')

hello marcus


In [19]:
my_function.__name__

'my_function'

In [20]:
marcus.__name__

NameError: name 'marcus' is not defined

In [77]:
class Outer:
    
    def __init__(Out, param_to_outer):
        Out.param_to_outer = param_to_outer
    
    def print_outer_exclusive(Out):
        print('Hy, im outer', Out.param_to_outer)
    
    class Inner(Outer):
        
        def __init__(self, param_to_inner):
            self.param_to_inner = param_to_inner
        
        def print_inner_exclusive(self):
            print('Hy, im inner', self.param_to_inner)
        
        def print_out_and_inner(self):
            print('Hy, im double printer. This is the inner arg: {}, and this is the out arg {}'.format(
            self.param_to_inner, Outer('test').print_outer_exclusive()))

In [78]:
Outer('olá').print_outer_exclusive()

Hy, im outer olá


In [79]:
Outer('olá').Inner('paraminner').print_inner_exclusive()

Hy, im inner paraminner


In [80]:
Outer('olá').Inner('paraminner').print_out_and_inner()

Hy, im outer test
Hy, im double printer. This is the inner arg: paraminner, and this is the out arg None
